In [80]:
import pandas as pd
import numpy as np

import splitter

from spellchecker import SpellChecker

import gensim
import nltk
from nltk.corpus import stopwords
from gensim.models import word2vec as w2v
LabeledSentence = gensim.models.doc2vec.LabeledSentence
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

import string

import os 
os.getcwd()

'/home/thanish/Competition/Zindi/Tech4MentalHealth/Notebook'

In [81]:
# Adding few words to the SpellChecker which it did not have before.
spell = SpellChecker()
spell.word_frequency.load_words(['bhang', 'google', 'counsellor', 'counselling', 'programme'])

In [82]:
# Reading the data
train_DF = pd.read_csv('../data/Train.csv', encoding='utf-8')
test_DF = pd.read_csv('../data/Test.csv', encoding='utf-8')

print(train_DF.shape)
train_DF.head()

(616, 3)


,ID,text,label
0,SUAVK39Z,I feel that it was better I dieAm happy,Depression
1,9JDAGUV3,Why do I get hallucinations?,Drugs
2,419WR1LQ,I am stresseed due to lack of financial suppor...,Depression
3,6UY7DX6Q,Why is life important?,Suicide
4,FYC0FTFB,How could I be helped to go through the depres...,Depression


In [83]:
# These are couple of words we want to correct
correction = {"’m" : " am",
              "\'m" : " am",
              "’ve": " have",
              "\'ve": " have",
              "’s" : " is",
              "\'s" : " is",
              "don’t": "do not",
              "don\'t": "do not",
              "didn’t": "did not",
              "didn\'t": "did not",
              "can’t": "can not",
              "can\'t": "can not",
              "aren’t": "are not",
              "aren\'t": "are not"}


def word_expansion(splitted_words):
    ''' 
    The above dictionary is used to expand the word 
    example: don't  will be expanded as do not
    '''
    for word in splitted_words:
        for correct in correction:
            if correct in word:
                corrrected_word = word.replace(correct, correction[correct])
                splitted_words[splitted_words.index(word)] = corrrected_word
    splitted_words = [" ".join(splitted_words)][0].split()
    return splitted_words

In [84]:
def compound_word_correction(words_list):
    '''
    This function corrects the words that are missppelled by having two words without a space.
    example: "drugsnow" will be replaced as "drugs now"
    '''
    corrected_words = [] 
    for word in words_list:
        corrected_words.append(word if splitter.split(word) == "" else splitter.split(word))   
    splitted_words = [val for sublist in corrected_words for val in sublist]
    return splitted_words

def ninja_word_correction(sent):
    '''
    This is similar to the compound word expansion. I used this for trial and error but did not 
    actually use it in the code.
    '''
    return wordninja.split(sent)


In [85]:
def special_character_correction(txt):
    '''
    This function basically is used to remove 
    some special characters that I found in the corpus. 
    '''
    if "…" in txt :
        txt = txt.replace("…", ".")

    elif "â€™" in txt:
        txt = txt.replace("â€™", "'")
    return txt

In [88]:
def spelling_correction(txt):
    '''
    This function is where most of the work happens. The text inputs from the train and test
    get's passed to this function to first remove the punctuation. The it get's fed to the 
    word expansion function, and then the spell check function. 
    When there are words present after being passed to spell.unknown(), it means that the spelling
    of the word is wrong which could be due to 2 reasons 
    1. Two correct words are written without a space as a single word
    2. Actually spelling of the word is wrong because of a typo.
    Such mispelled words are first past to the compound_word_correction() to check if expanding them 
    will correct it.
    If the length of the output words from the compound_word_correction)() is more than 2, it is sent to the 
    spell.correction(). This is because it's very unlikley to type 3 words together without a space between them.
    So if the output of the compound_word_correction() is more than 3 it means it's spelling error and not 
    compund word and compound_word_correction() was not able to split correctly. 
    
    '''
    
    txt = special_character_correction(txt)
    for j in '!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~':
        if (j in txt):
            txt = txt.replace(j, " " + j + " ")
    
    
    splitted_words = txt.lower().split()
    
    splitted_words = word_expansion(splitted_words)
    
    misspelled = spell.unknown(splitted_words)
    
    if len(misspelled)>0:
        for word in misspelled:
            print("Alert!!!!!!!!!!!!!!", word)
            word_index = splitted_words.index(word)
            cmp_word_correction = compound_word_correction(words_list = [word])
            corrected_word = [spell.correction(word)] if len(cmp_word_correction)>2 else cmp_word_correction
            splitted_words = splitted_words[:word_index] + corrected_word + splitted_words[(word_index+1):]
        corrected_sentence = [" ".join(splitted_words)][0] 
    else:
        corrected_sentence = [" ".join(splitted_words)][0] 
    return corrected_sentence


In [89]:
corrected = []
for i, sent in enumerate(train_DF.text):
    print(i, ":", sent)
    corrected.append(spelling_correction(txt = sent))
    print(i, ":", corrected[-1])
    print("")
train_DF['text'] = corrected

corrected = []
for i, sent in enumerate(test_DF.text):
    print(i, ":", sent)
    corrected.append(spelling_correction(txt = sent))
    print(corrected[-1])
    print("")
test_DF['text'] = corrected

0 : I feel that it was better I dieAm happy
Alert!!!!!!!!!!!!!! dieam
0 : i feel that it was better i die am happy

1 : Why do I get hallucinations?
1 : why do i get hallucinations ?

2 : I am stresseed due to lack of financial support in school
Alert!!!!!!!!!!!!!! stresseed
2 : i am stressed due to lack of financial support in school

3 : Why is life important?
3 : why is life important ?

4 : How could I be helped to go through the depression?
4 : how could i be helped to go through the depression ?

5 : What are the effects of depression to ones health
5 : what are the effects of depression to ones health

6 : Why is everything so hard to deal with in this life
6 : why is everything so hard to deal with in this life

7 :  I feel emotionally overwhelmed 
7 : i feel emotionally overwhelmed

8 : How to manage taking alcohol?
8 : how to manage taking alcohol ?

9 : Is heaven open for us who smoke bhang?
9 : is heaven open for us who smoke bhang ?

10 : How to survive without money?
10 :

166 : my sister often faints , this troubles me

167 : Hopeless and how to get out of it
167 : hopeless and how to get out of it

168 : I am depressed and want solitude
168 : i am depressed and want solitude

169 : Is withdrawal from people helpful from commiting suicide?
Alert!!!!!!!!!!!!!! commiting
169 : is withdrawal from people helpful from committing suicide ?

170 : i feel stressed
170 : i feel stressed

171 : Is there a peer counselling programme
171 : is there a peer counselling programme

172 : How to overcome alcoholism?
172 : how to overcome alcoholism ?

173 : How can I stop taking alcohol?
173 : how can i stop taking alcohol ?

174 : I hate my life
174 : i hate my life

175 : I have given up with everything in life 
175 : i have given up with everything in life

176 : Causes of suicide
176 : causes of suicide

177 :  im feeling stressed
177 : im feeling stressed

178 : Why is the larger percentage of youths involved in alcoholism
178 : why is the larger percentage of yout


434 : what is the cure for depression
434 : what is the cure for depression

435 : How can i feel better about myself
435 : how can i feel better about myself

436 : Whats the meaning of life?
436 : whats the meaning of life ?

437 : Why did my close relatives betray me?
437 : why did my close relatives betray me ?

438 : I feel very lonely that no one could understand
438 : i feel very lonely that no one could understand

439 : What is the benefit of alcohol
439 : what is the benefit of alcohol

440 : Should iI talk to my friends and peer counsellor about my depression ?
440 : should ii talk to my friends and peer counsellor about my depression ?

441 : how can I be helped to heal from depression
441 : how can i be helped to heal from depression

442 : I went through FGM when young even though I consented
442 : i went through fgm when young even though i consented

443 :  I feel like giving up in life
443 : i feel like giving up in life

444 : how to get over depression
444 : how to 

i am disappointed , i need a sense of closure

i lost hope

i feel unloved and uncared for

what are factors that leads to addiction of alcohol

Alert!!!!!!!!!!!!!! moderatly
is the advice of drinking moderately , prudent ?

how to limit myself from getting regular depression

i needed to know whether the alcohol releases stress

i was sad and did not want to go out

am i that ugly

how should i avoid alcohol intake ? what are the advantages of alcohol intake ?

i was pale and lost appetite

i am going through loss of a loved one . i am going through a breakup

what are the effects of bhang ?

depressed due to the loss of a relative

why me ? , why so ? , why now ?

feeling sad and lonely now feeling contentment and ability to withstand depression

can i overcome depression ?

how would i quite alcoholism

downplayed , self rejection

Alert!!!!!!!!!!!!!! commiting
what brings a person into committing suicide

does this make me feel any better ?

Alert!!!!!!!!!!!!!! undestands
feel no o

In [91]:
train_DF.to_csv('../data/train_corrected_3.csv', index = False)
test_DF.to_csv('../data/test_corrected_3.csv', index = False)

In [79]:
word = 'nott'

# trying to check the different methods of data correction methods available
spell.unknown([word]), spell.correction(word), compound_word_correction([word]), ninja_word_correction(word)

(set(), 'nott', ['not', 't'])